In [2]:
from SimPy.Simulation import *
import random
import numpy as np
import math
import read_data as rd

In [3]:
from pathlib import Path
import os
filenames = [str(Path(os.path.abspath('')).parent) + "\Data Collection\KevinYeData.txt",
            str(Path(os.path.abspath('')).parent) + "\Data Collection\dataviviandong.txt",
            str(Path(os.path.abspath('')).parent) + "\Data Collection\PatrickQData.txt",
            str(Path(os.path.abspath('')).parent) + "\Data Collection\TamaHoareData.txt"]
results = rd.ReadData(filenames)
AllArrivals = results[0]
ArrivalRate = 1/np.mean(AllArrivals)
AllServices = results[1]
ServiceRate = 1/np.mean(AllServices)
print("mean arrival rate:" + str(ArrivalRate) + " per second")
print("mean service rate:" + str(ServiceRate) + " per second")

mean arrival rate:0.008310517468785045 per second
mean service rate:0.008279591609419837 per second


In [4]:
def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return lower, upper

In [5]:
class Source(Process):
    """generate random arrivals"""
    def run(self, N):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = random.expovariate(ArrivalRate)
            yield hold, self, t

In [6]:
class Arrival(Process):
    """an arrival"""
    n=0 #class variable, number in system

    def run(self):
        Arrival.n +=1
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
        yield request, self, G.server
        yield hold, self, random.expovariate(ServiceRate)
        yield release, self, G.server
        Arrival.n -=1
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
        delay = now()-arrivetime
        G.delaymon.observe(delay)


In [7]:
class G:
    server = 'dummy'
    delaymon = 'Monitor' #observe time spent in system
    numbermon = "Monitor" #observe number customer in system
    busymon = 'Monitor' #observe time servers are busy

In [8]:
def model(N, maxtime, rvseed,K):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(K)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
    G.busymon = Monitor()


    Arrival.n = 0
    # simulate
    s = Source('Source')
    activate(s, s.run(N))
    simulate(until=maxtime)

    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    B = G.busymon.timeAverage()
    return W, L, B

The LAB Cafe is open from 7.30am to 3pm a total of 7.5 hours per day which is equivalent to 27000 seconds which would be the maxtime of each simulation.

In [9]:
## Experiment ----------------

for Kap in [3]:
    allW = []
    allL = []
    allB = []
    allLambdaEffective = []

    for k in range(50):
        seed = 123*k
        result = model(N=10000, maxtime=27000, rvseed=seed,K=Kap)
        allW.append(result[0])
        allL.append(result[1])
        allB.append(result[2])
        allLambdaEffective.append(result[1]/result[0])


    print("=================")
    print("When K=",Kap)
    print("    Estimate of W:", np.mean(allW))
    print("    Conf int of W:", conf(allW))
    print("    Estimate of L:", np.mean(allL))
    print("    Conf int of L:", conf(allL))
    print("    Estimate of B:", np.mean(allB))
    print("    Conf int of B:", conf(allB))
    print("    Estimate of LambdaEffective:", np.mean(allLambdaEffective))
    print("    Conf int of LambdaEffective:", conf(allLambdaEffective))

When K= 3
    Estimate of W: 124.33303596116097
    Conf int of W: (122.0551359913271, 126.61093593099484)
    Estimate of L: 1.0435912057915153
    Conf int of L: (1.016670225824624, 1.0705121857584066)
    Estimate of B: 0.641133566988506
    Conf int of B: (0.6306433427033502, 0.6516237912736618)
    Estimate of LambdaEffective: 0.00839529474026536
    Conf int of LambdaEffective: (0.008226262032210921, 0.0085643274483198)
